In [ ]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install accelerate
!apt install git-lfs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 72.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 94.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
torch.cuda.empty_cache()

from typing import Dict, Tuple
from datasets import list_datasets, load_dataset, DatasetDict,Dataset
from collections import Counter
from typing import List, Dict, Union, Callable, Any
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from pprint import pprint
import torch.nn as nn
import evaluate
from transformers import AutoTokenizer, T5ForConditionalGeneration, AutoConfig
from evaluate import TextClassificationEvaluator, Metric, EvaluationModuleInfo

In [ ]:
import transformers

print(transformers.__version__)

4.26.1


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device) 

cuda


## Preparing the dataset

In [ ]:
# dataset: DatasetDict = load_dataset("Sree1994/babylm_childstories")

ds_train = load_dataset("Sree1994/blm_strict_small", split="train")
ds_valid = load_dataset("Sree1994/blm_strict_small", split="valid")



raw_datasets = DatasetDict(
    {
        "train": ds_train,
        "valid": ds_valid
    }
)

raw_datasets

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating valid split:   0%|          | 0/5000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/20000 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/Sree1994___parquet/Sree1994--blm_strict_small-3934cc01075622aa/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 20000
    })
    valid: Dataset({
        features: ['text'],
        num_rows: 5000
    })
})

In [ ]:
model_checkpoint = "t5-small"

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, DataCollatorForSeq2Seq, Trainer, TrainingArguments,AutoTokenizer

tokenizer = T5Tokenizer.from_pretrained(model_checkpoint)
context_length = 128
vocab_size = tokenizer.vocab_size

/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [ ]:
train_encoded_texts = tokenizer(
    raw_datasets['train']['text'],
    padding='max_length',
    truncation=True,
    max_length=512,  # set a fixed length for the input and target sequences
    return_tensors='pt',
    add_special_tokens=True,
)

In [ ]:
valid_encoded_texts = tokenizer(
    raw_datasets['valid']['text'],
    padding='max_length',
    truncation=True,
    max_length=512,  # set a fixed length for the input and target sequences
    return_tensors='pt',
    add_special_tokens=True,
)

In [ ]:
config = AutoConfig.from_pretrained(
    "t5-small",
    vocab_size=len(tokenizer),
    random_init=True
)
print(len(tokenizer))
model = T5ForConditionalGeneration(config).to(device)
model.init_weights()
model_size = sum(t.numel() for t in model.parameters())
print(f"T5 size: {model_size/1000**2:.1f}M parameters")

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--t5-small/snapshots/3479082dc36f8a4730936ef1c9b88cd8b0835c53/config.json
Model config T5Config {
  "_name_or_path": "t5-small",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
   

32100
T5 size: 60.5M parameters


In [ ]:
from torch.utils.data import TensorDataset
train_dataset = TensorDataset(
    train_encoded_texts['input_ids'],
    train_encoded_texts['attention_mask'],
    train_encoded_texts['input_ids'],
    train_encoded_texts['attention_mask'],
)

valid_dataset = TensorDataset(
    valid_encoded_texts['input_ids'],
    valid_encoded_texts['attention_mask'],
    valid_encoded_texts['input_ids'],
    valid_encoded_texts['attention_mask'],
)

In [ ]:
data_collator = lambda data: {'input_ids': torch.stack([item[0] for item in data]),
                              'attention_mask': torch.stack([item[1] for item in data]),
                              'decoder_input_ids': torch.stack([item[2] for item in data]),
                              'decoder_attention_mask': torch.stack([item[3] for item in data])}


In [ ]:
from transformers import Trainer, TrainingArguments

args = TrainingArguments(
    output_dir="./babylm/output",
    do_train=True,
    do_eval=True,
    do_predict=True,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="epoch",
    eval_steps=5_000,
    logging_steps=5_000,
    gradient_accumulation_steps=8,
    num_train_epochs=10,
    weight_decay=0.001,
    warmup_steps=1_000,
    lr_scheduler_type="cosine",
    learning_rate=4e-4,
    save_steps=500,
    #fp16=True,
    push_to_hub=False,
    
)

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
def compute_loss(model, inputs, return_outputs=False):
    labels = inputs.pop("decoder_input_ids")
    outputs = model(**inputs, labels=labels)
    loss = outputs.loss
    return (loss, outputs) if return_outputs else loss

trainer.compute_loss = compute_loss

In [ ]:
trainer.train()

***** Running training *****
  Num examples = 20000
  Num Epochs = 10
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 8
  Total optimization steps = 3120
  Number of trainable parameters = 60492288


Epoch,Training Loss,Validation Loss


In [ ]:
trn.training_loss

In [ ]:
valid = val.get("test_loss")
torch.exp(torch.tensor(valid))